In [1]:
pip install tqdm

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

#0. Initial Setting

In [3]:
import numpy as np
from scipy.special import logsumexp
from scipy.special import softmax
import warnings
warnings.filterwarnings("ignore")

theta1 = np.array([2,-1])#np.array([[[4.57,3.41],[3.86,3.18]],[[2.69,2.75],[3.58,4.57]]]) #s,a1,a2

theta23 = np.array([0.9,0.5,0.5,0.5,0.5,0.5,0.5,0.1,0.9,0.2,0.8,0.2,0.8,0.2,0.8,0.1])



TimePeriod= 150
SamplePath = 5000
num_discrete = 100

z_old = np.arange(2,dtype=int) #observation
a1_old = np.arange(2,dtype = int)
a2_old = np.arange(2,dtype = int)
x_old = np.linspace(0,1,num_discrete) #belief

#1. Generate Dynamic Part and Policy Part

In [4]:
def Dynamic(theta23):
  
  trans = np.ones([2,2,2,2])      #s_t,a1,a2,s_{t+1}
  # trans[0,:,:,0] = trans[1,:,:,1] = 1-theta23[1]
  # trans[0,:,:,1] = trans[1,:,:,0] = theta23[1]

  # trans[0,0,0,0] = trans[1,1,1,1] = theta23[0]
  # trans[0,0,0,1] = trans[1,1,1,0] = 1-theta23[0]
  trans[:,:,:,0] = np.reshape(theta23[0:8],[2,2,2])
  trans[:,:,:,1] = 1 - trans[:,:,:,0]


  observe = np.ones([2,2,2,2]) #a1,a2,s_{t+1},z_{t+1}
  # observe[:,:,0,0] = observe[:,:,1,1] = 1-theta23[9]#[3]
  # observe[:,:,0,1] = observe[:,:,1,0] = theta23[9]#[3]
  # observe[0,0,0,0] = observe[1,1,1,1] = theta23[8]#[2]
  # observe[0,0,0,1] = observe[1,1,1,0] = 1-theta23[8]#[2]
  observe[:,:,:,0] = np.reshape(theta23[8:16],[2,2,2])
  observe[:,:,:,1] = 1 - observe[:,:,:,0]
  return trans,observe
#print(Dynamic(theta23))

def Reward(theta1):
  rw = np.ones([2,2,2])#s,a1,a2
  rw = theta1[1]*rw
  rw[0,0,0] = rw[1,1,1] = theta1[0]
  return rw
#print(Reward(theta1))

def SigmaLambda(z_old,x_old,a1_old,a2_old,theta23= theta23,num_discrete=num_discrete,T = None): #xt,a1_t,a2_t,z_{t+1}
  trans,observe = Dynamic(theta23)  #s_t,a1,a2,s_{t+1} #a1,a2,s_{t+1},z_{t+1}
  if T==None: #generate function for Q function iteration
      x_new = np.zeros([num_discrete,2,2,2])
      sigma = np.zeros([num_discrete,2,2,2])
      x_old = np.vstack([x_old,1-x_old]).T

      for a1 in a1_old:#range(2):
        for a2 in a2_old:#range(2):
          for z_new in z_old:#range(2):
            x_new_tep = x_old.dot(trans[:,a1,a2,0]*observe[a1,a2,0,z_new])
            sigma[:,a1,a2,z_new] = x_new_tep + x_old.dot(trans[:,a1,a2,1]*observe[a1,a2,1,z_new])
            x_new[:,a1,a2,z_new] = x_new_tep/sigma[:,a1,a2,z_new]
      belief_f = np.floor(x_new*(num_discrete-1))/(num_discrete-1)
      belief_c = np.ceil(x_new*(num_discrete-1))/(num_discrete-1)
      iterpolate = np.zeros(belief_f.shape)
      iterpolate[np.where((belief_f-belief_c)!=0)] = (x_new-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]
  elif T==1: #generate 1 period data for data generation
      x_new = np.zeros(x_old.shape)
      sigma = np.zeros(x_old.shape)
      #x_old = np.vstack([x_old,1-x_old]).T

      x_new_tep = (x_old *trans[0,a1_old,a2_old,0] + (1-x_old)*trans[1,a1_old,a2_old,0] )* observe[a1_old,a2_old,0,z_old]
      sigma = x_new_tep + (x_old *trans[0,a1_old,a2_old,1] + (1-x_old)*trans[1,a1_old,a2_old,1] )* observe[a1_old,a2_old,1,z_old]
      x_new = x_new_tep/sigma

      belief_f = np.floor(x_new*(num_discrete-1))/(num_discrete-1)
      belief_c = np.ceil(x_new*(num_discrete-1))/(num_discrete-1)
      iterpolate = np.zeros(belief_f.shape)
      iterpolate[np.where((belief_f-belief_c)!=0)] = (x_new-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]
  else:      #generate whole belief for recover process
      x_new = np.zeros(a1_old.shape) # T, N
      sigma = np.zeros([T-1,z_old.shape[1]]) # T-1 ,N
      x_new[0,:] = x_old
      #x_old = np.vstack([x_old,1-x_old]).T
      for t in range(1,T):
        a1 = a1_old[t-1,:]
        a2 = a2_old[t-1,:]
        z = z_old[t,:]
        x_new_tep = (x_new[t-1,:] *trans[0,a1,a2,0] + (1-x_new[t-1,:])*trans[1,a1,a2,0] )* observe[a1,a2,0,z]
        sigma[t-1,:] = x_new_tep + (x_new[t-1,:] *trans[0,a1,a2,1] + (1-x_new[t-1,:])*trans[1,a1,a2,1] )* observe[a1,a2,1,z]
        x_new[t,:] = x_new_tep/sigma[t-1,:]

      belief_f = np.floor(x_new*(num_discrete-1))/(num_discrete-1)
      belief_c = np.ceil(x_new*(num_discrete-1))/(num_discrete-1)
      iterpolate = np.zeros(belief_f.shape)
      iterpolate[np.where((belief_f-belief_c)!=0)] = (x_new-belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]      

  return sigma,x_new,[iterpolate,np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))]
#print(SigmaLambda()[0][:,0,0,0])



def ValueIteration(theta1,sigma,xiterpolate, num_discrete=num_discrete,beta = 0.9,
                   gamma = 0.5772,thread_inner = 1e-5,thread_outer = 1e-5): #xt, a1 or xt,a2
  rw = Reward(theta1)
  iterpolate,belief_f,belief_c = xiterpolate[0],xiterpolate[1],xiterpolate[2]
  x_old = np.linspace(0,1,num=num_discrete)
  #x_old = np.vstack([x_old,1-x_old]).T

  pi1_new = 0.5*np.ones([num_discrete,2]) #xt,a1
  # Q1_new = np.zeros([num_discrete,2]) #xt,a2
  # Q2_new = np.zeros([num_discrete,2])

  pi2_old = pi1_new.copy()
  count = 0
  #pi1_old = np.zeros([2,2])
  for ol in range(1000):
    pi1_old = pi1_new.copy()
    
    Q1_new = np.zeros([num_discrete,2]) #xt,a2
    Q2_new = np.zeros([num_discrete,2])

    for il in range(1000):
      Q2_old = Q2_new.copy()
      for a2 in range(2):
        #a1 = 0
        it0,bf0,bc0 = iterpolate[:,0,a2,:],belief_f[:,0,a2,:],belief_c[:,0,a2,:]
        V2_a10 =  np.stack([it0*Q2_old[bf0,0] + (1-it0)*Q2_old[bc0,0],
                            it0*Q2_old[bf0,1] + (1-it0)*Q2_old[bc0,1]],axis=2)
        Q2_tepa10 = x_old * rw[0,0,a2] + (1-x_old) * rw[1,0,a2] + beta * np.sum(sigma[:,0,a2,:]*(gamma+logsumexp(V2_a10,axis=2)),axis=1)

        #a1=1
        it1,bf1,bc1 = iterpolate[:,1,a2,:],belief_f[:,1,a2,:],belief_c[:,1,a2,:]
        V2_a11 =  np.stack([it1*Q2_old[bf1,0] + (1-it1)*Q2_old[bc1,0],
                            it1*Q2_old[bf1,1] + (1-it1)*Q2_old[bc1,1]],axis=2)
        Q2_tepa11 = x_old * rw[0,1,a2] + (1-x_old) * rw[1,1,a2] + beta * np.sum(sigma[:,1,a2,:]*(gamma+logsumexp(V2_a11,axis=2)),axis=1)
          
        Q2_new[:,a2] = pi1_old[:,0] *Q2_tepa10 + pi1_old[:,1] * Q2_tepa11
        #print(it0)
      #print(il,np.max(np.abs((Q2_new-Q2_old)/Q2_old)),np.max(np.abs(Q2_old)))
      if np.max(np.abs((Q2_new-Q2_old)/Q2_old)) <thread_inner: 
        #print('Q2',il,np.max(np.abs((Q2_new-Q2_old)/Q2_old)))
        break

    pi2_new = softmax(Q2_new,axis=1)
    #if np.max(np.abs((pi2_old-pi2_new)/pi2_old)) <thread_outer:
    count = np.max(np.abs((pi2_old-pi2_new)/pi2_old))


    pi2_old = pi2_new.copy()
    for il in range(1000):
      Q1_old = Q1_new.copy()
      for a1 in range(2):
        #a2 = 0
        it0,bf0,bc0 = iterpolate[:,a1,0,:],belief_f[:,a1,0,:],belief_c[:,a1,0,:]
        V1_a20 =  np.stack([it0*Q1_old[bf0,0] + (1-it0)*Q1_old[bc0,0],
                            it0*Q1_old[bf0,1] + (1-it0)*Q1_old[bc0,1]],axis=2)
        Q1_tepa20 = x_old * rw[0,a1,0] + (1-x_old) * rw[1,a1,0] + beta * np.sum(sigma[:,a1,0,:]*(gamma+logsumexp(V1_a20,axis=2)),axis=1)

        #a2=1
        it1,bf1,bc1 = iterpolate[:,a1,1,:],belief_f[:,a1,1,:],belief_c[:,a1,1,:]
        V1_a21 =  np.stack([it1*Q1_old[bf1,0] + (1-it1)*Q1_old[bc1,0],
                            it1*Q1_old[bf1,1] + (1-it1)*Q1_old[bc1,1]],axis=2)
        Q1_tepa21 = x_old * rw[0,a1,1] + (1-x_old) * rw[1,a1,1] + beta * np.sum(sigma[:,a1,1,:]*(gamma+logsumexp(V1_a21,axis=2)),axis=1)
          
        Q1_new[:,a1] = pi2_old[:,0] *Q1_tepa20 + pi2_old[:,1] * Q1_tepa21
      #print(il,np.max(np.abs((Q1_new-Q1_old)/Q1_old)),np.max(np.abs(Q1_old)))

      if np.max(np.abs((Q1_new-Q1_old)/Q1_old)) <thread_inner:
        #print('Q1',il,np.max(np.abs((Q1_new-Q1_old)/Q1_old)))
        break

    pi1_new = softmax(Q1_new,axis=1)
    
    #print('pi1 pi2',ol,np.max(np.abs((pi1_old-pi1_new)/pi1_old)),count)

    if np.max(np.abs((pi1_old-pi1_new)/pi1_old)) <thread_outer and count<thread_outer:
      #print(ol,np.max(np.abs((pi1_old-pi1_new)/pi1_new)))
      break
  return Q1_new,Q2_new,pi1_new,pi2_new

# sigma,_,xiterpolate= SigmaLambda(z_old,x_old,a1_old,a2_old,theta23=theta23,num_discrete=num_discrete)
# Q1,Q2,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)

Check Q value function works (doesn't depend on discrete intervals) 

In [5]:
# import matplotlib.pyplot as plt
# from tqdm import tqdm
# discrete_tol = 10*(1+np.arange(20))
# Q1_plot = []
# Q2_plot = []
# pi1_plot = []
# pi2_plot = []
# for num_disc in tqdm(discrete_tol):
#   x_old_test = np.linspace(0,1,num_disc)
#   sigma,_,xiterpolate= SigmaLambda(z_old,x_old_test,a1_old,a2_old,theta23=theta23,num_discrete=num_disc)

#   Q1,Q2,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate, num_discrete=num_disc)
#   Q1_plot.append(Q1)
#   Q2_plot.append(Q2)
#   pi1_plot.append(pi1)
#   pi2_plot.append(pi2)

# for i in range(len(discrete_tol)):
#   plt.plot(np.linspace(0,1,discrete_tol[i]),pi2_plot[i][:,0],np.linspace(0,1,discrete_tol[i]),pi1_plot[i][:,0])

#2. Data Generation Process

In [6]:
##################################Global Parameter#################################
# theta1 = np.array([4.57,2.5])#np.array([[[4.57,3.41],[3.86,3.18]],[[2.69,2.75],[3.58,4.57]]]) #s,a1,a2
# theta23 = np.array([0.9,0.5,0.9,0.2])
# TimePeriod= 150
# SamplePath = 300
# num_discrete = 100
# z_old = np.arange(2,dtype=int) #observation
# a1_old = np.arange(2,dtype = int)
# a2_old = np.arange(2,dtype = int)
# x_old = np.linspace(0,1,num_discrete) #belief
###################################################################################
from tqdm import tqdm
def DataGeneration(theta1 = theta1,theta23=theta23,T=TimePeriod,N=SamplePath,Nsub = 100,num_discrete=num_discrete):
  HdState = np.zeros([T,N],dtype= int)    #hidden state
  ObState = np.zeros([T,N],dtype = int) #observation state
  belief = np.zeros([T,N])                #belief x
  action1 = np.zeros([T,N],dtype = int)   #agent1's action
  action2 = np.zeros([T,N],dtype = int)   #agent2's action

  HdState[0,:] = np.random.randint(0,2,size=N)
  belief[0,:] = np.kron(np.ones(Nsub),np.linspace(0,1,int(N/Nsub)))#np.random.uniform(size=N)
  ObState[0,:] = np.int_(np.random.random(size=N)>belief[0,:])
      

  trans,observe = Dynamic(theta23)
  sigma,_,xiterpolate= SigmaLambda(z_old,x_old,a1_old,a2_old,theta23=theta23,num_discrete=num_discrete)
  _,_,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)
  for t in tqdm(range(T)):
    # iterpolate belief
    belief_f = np.floor(belief[t,:] *(num_discrete-1))/(num_discrete-1)
    belief_c = np.ceil(belief[t,:] *(num_discrete-1))/(num_discrete-1)
    iterpolate = np.zeros(belief_f.shape)
    iterpolate[np.where((belief_f-belief_c)!=0)] = (belief[t,:] -belief_c)[np.where((belief_f-belief_c)!=0)]/(belief_f-belief_c)[np.where((belief_f-belief_c)!=0)]
    belief_fint,belief_cint = np.int_(belief_f*(num_discrete-1)),np.int_(belief_c*(num_discrete-1))

    # generate new action for both agents
    action_mat1 = np.random.random(size=N)
    p1 = iterpolate*pi1[belief_fint,0] + (1-iterpolate)*pi1[belief_cint,0]
    action1[t,:] = np.int_(p1<=action_mat1)
    
    action_mat2 = np.random.random(size=N)
    p2 = iterpolate*pi2[belief_fint,0] + (1-iterpolate)*pi2[belief_cint,0]
    action2[t,:] = np.int_(p2<=action_mat2)
    
    if t!=T-1:
      # generate new hidden state and observation state
      state_mat1 = np.random.random(size=N)
      HdState[t+1,:] = np.int_(trans[HdState[t,:],action1[t,:],action2[t,:],0]<=state_mat1)
      state_mat2 = np.random.random(size=N)
      ObState[t+1,:] = np.int_(observe[action1[t,:],action2[t,:],HdState[t+1,:],0]<=state_mat2)

      # update new belief
      _,belief[t+1,:],_ = SigmaLambda(ObState[t+1,:] ,belief[t,:],action1[t,:],action2[t,:],theta23=theta23,num_discrete=num_discrete,T=1)
  return ObState, action1,action2,belief,HdState


ObState, action1,action2,belief,HdState= DataGeneration()
#haha = SigmaLambda()

100%|██████████| 150/150 [00:00<00:00, 679.77it/s]


#3.1 Recover Dynamic Part

In [7]:
theta23

array([0.9, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1, 0.9, 0.2, 0.8, 0.2, 0.8,
       0.2, 0.8, 0.1])

In [8]:
from scipy.optimize import minimize
ini_x0 = 0.8*theta23
print('start from:',ini_x0 )

def func1(theta23,ObState=ObState, action1=action1,action2 = action2,x0=belief[0,:],T=TimePeriod):
  sigma,_,_ = SigmaLambda(ObState,x0,action1,action2,theta23= theta23,num_discrete=num_discrete,T = TimePeriod)
  res = -np.sum(np.log(sigma))
  print(theta23,res)
  return res

res1 = minimize(func1,ini_x0, bounds=[[0.00001,0.99999],[0.00001,0.99999],[0.00001,0.99999],[0.00001,0.99999],
                                      [0.00001,0.99999],[0.00001,0.99999],[0.00001,0.99999],[0.00001,0.99999],
                                      [0.00001,0.99999],[0.00001,0.99999],[0.00001,0.99999],[0.00001,0.99999],
                                      [0.00001,0.99999],[0.00001,0.99999],[0.00001,0.99999],[0.00001,0.99999]])
#trans_est = np.vstack([res1.x,1-res1.x]).T.reshape([2,2,2,2])
print(res1)

start from: [0.72 0.4  0.4  0.4  0.4  0.4  0.4  0.08 0.72 0.16 0.64 0.16 0.64 0.16
 0.64 0.08]
[0.72 0.4  0.4  0.4  0.4  0.4  0.4  0.08 0.72 0.16 0.64 0.16 0.64 0.16
 0.64 0.08] 451057.83722090215
[0.72000001 0.4        0.4        0.4        0.4        0.4
 0.4        0.08       0.72       0.16       0.64       0.16
 0.64       0.16       0.64       0.08      ] 451057.83543815755
[0.72       0.40000001 0.4        0.4        0.4        0.4
 0.4        0.08       0.72       0.16       0.64       0.16
 0.64       0.16       0.64       0.08      ] 451057.8371120851
[0.72       0.4        0.40000001 0.4        0.4        0.4
 0.4        0.08       0.72       0.16       0.64       0.16
 0.64       0.16       0.64       0.08      ] 451057.8371117913
[0.72       0.4        0.4        0.40000001 0.4        0.4
 0.4        0.08       0.72       0.16       0.64       0.16
 0.64       0.16       0.64       0.08      ] 451057.83708064124
[0.72       0.4        0.4        0.4        0.40000001 0.4
 

#3.2 Recover Reward Part

In [9]:
theta1.reshape(-1)

array([ 2, -1])

In [10]:
ini_x0 = 0.85*theta1#[0]
print('start from:',ini_x0 )
sigma,_,xiterpolate= SigmaLambda(z_old,x_old,a1_old,a2_old,theta23=res1.x,num_discrete=num_discrete)
_,_,beliefIP = SigmaLambda(ObState,belief[0,:],action1,action2,theta23= res1.x,num_discrete=num_discrete,T = TimePeriod)

def func2(theta1,sigma = sigma,xiterpolate=xiterpolate,beliefIP=beliefIP,action1 = action1,action2 = action2,T=TimePeriod):
  #theta1 = theta1.reshape([2,2,2])
  #_,_,pi1,pi2 = ValueIteration(np.array([theta1,1.5]),sigma,xiterpolate)
  _,_,pi1,pi2 = ValueIteration(theta1,sigma,xiterpolate)
  p1 = beliefIP[0] *pi1[beliefIP[1],action1] + (1-beliefIP[0]) *pi1[beliefIP[2],action1]
  p2 = beliefIP[0] *pi2[beliefIP[1],action2] + (1-beliefIP[0]) *pi2[beliefIP[2],action2]
  res = - np.sum(np.log(p1)) -np.sum(np.log(p2))
  print(theta1,res)
  return res

res2 = minimize(func2,ini_x0,bounds = [(1e-6,100),(-100,-1e-6)])#method='Nelder-Mead')
print(res2)

start from: [ 1.7  -0.85]
[ 1.7  -0.85] 420063.7806449019
[ 1.70000001 -0.85      ] 420063.78013713623
[ 1.7        -0.84999999] 420063.7811526685
[ 100. -100.] 24204478.718469106
[ 100.00000001 -100.        ] 24204478.71967764
[100.         -99.99999999] 24204478.717260554
[ 9.45392173 -8.67096988] 2196603.005761889
[ 9.45392174 -8.67096988] 2196603.0069738072
[ 9.45392173 -8.67096987] 2196603.0045499736
[ 2.35031432 -1.50593759] 461523.2646808785
[ 2.35031433 -1.50593759] 461523.2657017051
[ 2.35031432 -1.50593758] 461523.2636600528
[ 1.88445387 -1.03604884] 410254.7295394541
[ 1.88445388 -1.03604884] 410254.7294925749
[ 1.88445387 -1.03604883] 410254.7295863335
[ 1.90321566 -1.05497291] 410155.2379804551
[ 1.90321567 -1.05497291] 410155.2379744761
[ 1.90321566 -1.0549729 ] 410155.2379864337
[ 1.90595834 -1.05773911] 410153.6559018508
[ 1.90595835 -1.05773911] 410153.6559018559
[ 1.90595834 -1.0577391 ] 410153.65590184636
[ 1.90595599 -1.05773706] 410153.6559009278
[ 1.905956   -1.05

In [11]:
print(func2(theta1))
sigma,_,xiterpolate= SigmaLambda(z_old,x_old,a1_old,a2_old,theta23=theta23,num_discrete=num_discrete)
_,_,beliefIP = SigmaLambda(ObState,belief[0,:],action1,action2,theta23= theta23,num_discrete=num_discrete,T = TimePeriod)
print(func1(theta23), func2(theta1,sigma = sigma,xiterpolate=xiterpolate,beliefIP=beliefIP))

[ 2 -1] 410225.3298016177
410225.3298016177
[0.9 0.5 0.5 0.5 0.5 0.5 0.5 0.1 0.9 0.2 0.8 0.2 0.8 0.2 0.8 0.1] 393640.1379387274
[ 2 -1] 409472.9950980349
393640.1379387274 409472.9950980349


In [12]:
# ini_x0 = 0.8*theta1[0].reshape(-1)
# print('start from:',ini_x0 )
# sigma,_,xiterpolate= SigmaLambda(z_old,x_old,a1_old,a2_old,theta23=theta23,num_discrete=num_discrete)
# _,_,beliefIP = SigmaLambda(ObState,belief[0,:],action1,action2,theta23= theta23,num_discrete=num_discrete,T = TimePeriod)

# def func2(theta11,theta12= theta1[1],sigma = sigma,xiterpolate=xiterpolate,beliefIP=beliefIP,action1 = action1,action2 = action2,T=TimePeriod):
#   #theta1 = theta1.reshape([2,2,2])
#   _,_,pi1,pi2 = ValueIteration(np.array([theta11,theta12]),sigma,xiterpolate)
#   p1 = beliefIP[0] *pi1[beliefIP[1],action1] + (1-beliefIP[0]) *pi1[beliefIP[2],action1]
#   p2 = beliefIP[0] *pi2[beliefIP[1],action2] + (1-beliefIP[0]) *pi2[beliefIP[2],action2]
#   res = - np.sum(np.log(p1)) -np.sum(np.log(p2))
#   print(theta11,res)
#   return res

# res2 = minimize(func2,ini_x0,bounds = [(1e-6,100)])#,(1e-6,100)])#method='Nelder-Mead')
# print(res2)

#4. Complete Cases

In [13]:
def DynamicComplete(theta23):
  
  trans = np.ones([2,2,2,2])      #s_t,a1,a2,s_{t+1}
  trans[:,:,:,0] = np.reshape(theta23,[2,2,2])
  trans[:,:,:,1] = 1 - trans[:,:,:,0]

  return trans
#print(Dynamic([0.9,0.5,0.5,0.5,0.5,0.5,0.5,0.1]))
def RewardComplete(theta1):
  rw = np.ones([2,2,2])#s,a1,a2
  rw = theta1[1]*rw
  rw[0,0,0] = rw[1,1,1] = theta1[0]
  return rw

def ValueIterationComplete(theta1 = theta1, theta23= theta23, beta = 0.9,gamma = 0.5772,thread_inner = 1e-5,thread_outer = 1e-5):
  trans = DynamicComplete(theta23)
  rw = RewardComplete(theta1)
  pi1_new = 0.5*np.ones([2,2]) #s,a1
  Q1_new = np.zeros([2,2]) #s,a2
  Q2_new = np.zeros([2,2])

  #pi1_old = np.zeros([2,2])
  for ol in range(1000):
    pi1_old = pi1_new.copy()
    for il in range(1000):
      Q2_old = Q2_new.copy()
      Q2_tep = rw + beta * trans.dot(gamma + logsumexp(Q2_old,axis=1))
      Q2_new[:,0] = np.sum(pi1_old * Q2_tep[:,:,0],axis=1)
      Q2_new[:,1] = np.sum(pi1_old * Q2_tep[:,:,1],axis=1)
      if np.max(np.abs((Q2_new-Q2_old)/Q2_old)) <thread_inner: 
        #print(il,np.max(np.abs((Q2_new-Q2_old)/Q2_old)))
        break
    pi2_new = softmax(Q2_new,axis=1)
    pi2_old = pi2_new.copy()
    for il in range(1000):
      Q1_old = Q1_new.copy()
      Q1_tep = rw + beta * trans.dot(gamma + logsumexp(Q1_old,axis=1))
      Q1_new[:,0] = np.sum(pi2_old * Q1_tep[:,0,:],axis=1)
      Q1_new[:,1] = np.sum(pi2_old * Q1_tep[:,1,:],axis=1)
      if np.max(np.abs((Q1_new-Q1_old)/Q1_old)) <thread_inner:
        #print(il,np.max(np.abs((Q1_new-Q1_old)/Q1_old)))
        break     
    pi1_new = softmax(Q1_new,axis=1)
    
    #print(ol,np.max(np.abs((pi1_old-pi1_new)/pi1_new)))
    if np.max(np.abs((pi1_old-pi1_new)/pi1_new)) <thread_outer:
      #print(ol,np.max(np.abs((pi1_old-pi1_new)/pi1_new)))
      break
  return Q1_new,Q2_new,pi1_new,pi2_new
#ValueIteration(theta1 = theta1, theta23= theta23[0:2])

In [14]:
theta23_complete = theta23[0:8]
print(DynamicComplete(theta23_complete))

[[[[0.9 0.1]
   [0.5 0.5]]

  [[0.5 0.5]
   [0.5 0.5]]]


 [[[0.5 0.5]
   [0.5 0.5]]

  [[0.5 0.5]
   [0.1 0.9]]]]


In [15]:
from scipy.optimize import minimize
ini_x0 = 0.8*theta23_complete
print('start from:',ini_x0 )
def func1Complete(theta23,state = ObState,# HdState,#
          action1=action1,action2 = action2,T=TimePeriod):
  trans = DynamicComplete(theta23)
  #trans = np.vstack([trans,1-trans]).T.reshape([2,2,2,2]) #st,a1,a2,st+1
  return -np.sum(np.log(trans[state[0:-1,:],action1[0:-1,:],action2[0:-1,:],state[1:T,:]]))

res1 = minimize(func1Complete,ini_x0, bounds=[[0.00001,0.99999],[0.00001,0.99999],[0.00001,0.99999],[0.00001,0.99999],
                                      [0.00001,0.99999],[0.00001,0.99999],[0.00001,0.99999],[0.00001,0.99999]])#trans[:,:,:,0].reshape(-1), bounds=[[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]],
                #method='Nelder-Mead')
print(res1)

start from: [0.72 0.4  0.4  0.4  0.4  0.4  0.4  0.08]
      fun: 395587.9135993764
 hess_inv: <8x8 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.41327439, -4.99421731, -3.7427526 , -2.7474016 , -1.990702  ,
       -1.82189979,  6.71716407, 11.78705133])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 180
      nit: 16
   status: 0
  success: True
        x: array([0.8015941 , 0.49642948, 0.49749887, 0.32732097, 0.67041522,
       0.49910962, 0.50625644, 0.19658558])


In [16]:
theta1

array([ 2, -1])

In [17]:
ini_x0 = 0.85*theta1#[0]
print('start from:',ini_x0 )

def func2Complete(theta1,theta23 = res1.x,state = ObState,# HdState,#
          action1 = action1,action2 = action2):
  trans = DynamicComplete(theta23)
  rw = RewardComplete(theta1)
  _,_,pi1,pi2 = ValueIterationComplete(theta1=theta1,theta23=theta23)
  res = - np.sum(np.log(pi1[state,action1])) -np.sum(np.log(pi2[state,action2]))
  print(theta1,res)
  return res

res2 = minimize(func2Complete,ini_x0,bounds = [(1e-6,100),(-100,-1e-6)])
print(res2)

start from: [ 1.7  -0.85]
[ 1.7  -0.85] 509033.4047642966
[ 1.70000001 -0.85      ] 509033.4050496995
[ 1.7        -0.84999999] 509033.4044788906
[ 1.e-06 -1.e-06] 1039720.1796189577
[ 1.01e-06 -1.00e-06] 1039720.1766628532
[ 1.0e-06 -9.9e-07] 1039720.1825750635
[ 1.63571089 -0.81785546] 507050.7884910162
[ 1.6357109  -0.81785546] 507050.78861402615
[ 1.63571089 -0.81785545] 507050.7883680087
[ 1.58701272 -0.79350805] 506636.5717406318
[ 1.58701273 -0.79350805] 506636.57172940095
[ 1.58701272 -0.79350804] 506636.5717518622
[ 1.59108692 -0.79554494] 506633.2589240406
[ 1.59108693 -0.79554494] 506633.2589244194
[ 1.59108692 -0.79554493] 506633.2589236655
[ 1.59095388 -0.79547922] 506633.25517591217
[ 1.59095389 -0.79547922] 506633.25517591153
[ 1.59095388 -0.79547921] 506633.2551759096
[ 1.5909542  -0.79547867] 506633.25517590734
[ 1.59095421 -0.79547867] 506633.2551759097
[ 1.5909542  -0.79547866] 506633.2551759081
      fun: 506633.25517590734
 hess_inv: <2x2 LbfgsInvHessProduct with d

In [18]:
print(func2Complete(theta1=theta1,theta23=res1.x),func2Complete(theta1=theta1,theta23=theta23_complete))

[ 2 -1] 535603.0977830692
[ 2 -1] 535603.0977830694
535603.0977830692 535603.0977830694


In [3]:
import numpy as np
import numpy as np
def AIC(loglike,k,n):
  return -2*loglike + 2*k #+ 2*k*(k+1)/(n-k-1)

def BIC(loglike,k,n):
  return -2*loglike + k*np.log(n) 

print(AIC(-393622.73171 - 410153.65590,18,5000*150))
print(AIC(-395587.91359 - 506633.25517,10,5000*150))
print(BIC(-393622.73171 - 410153.65590,18,5000*150))
print(BIC(-395587.91359 - 506633.25517,10,5000*150))


1607588.7752200002
1804462.33752
1607796.2761327394
1804577.6158048552


In [4]:
(395587.91359 + 506633.25517 -393622.73171 - 410153.65590)/(393622.73171 + 410153.65590)

0.12247782177667843